# Inference using Falcon 7B - Uncensored for generating toxic data

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers accelerate einops sentencepiece

Clone the required repositories to be able to install AutoGPTQ

In [ ]:
!rm -rf AutoGPTQ
!git clone https://github.com/PanQiWei/AutoGPTQ.git && cd AutoGPTQ
!git clone https://github.com/oobabooga/text-generation-webui

Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 4688, done.
remote: Counting objects: 100% (1430/1430), done.
remote: Compressing objects: 100% (400/400), done.
remote: Total 4688 (delta 1137), reused 1245 (delta 1028), pack-reused 3258
Receiving objects: 100% (4688/4688), 8.13 MiB | 23.45 MiB/s, done.
Resolving deltas: 100% (3043/3043), done.
fatal: destination path 'text-generation-webui' already exists and is not an empty directory.


In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


NOTE: Use the pip install code below if the inference speed slows down to download the latest version for AutoGPTQ

In [ ]:
!pip install "git+https://github.com/PanQiWei/AutoGPTQ.git@v0.4.2"

Try to install the version below if the inference speed slows down.

In [ ]:
!pip install -q auto-gptq==0.4.2
from auto_gptq import AutoGPTQForCausalLM

In [ ]:
!pip install auto_gptq

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#Load the model:

In [ ]:
# import the required libraries
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

In [ ]:
model_path = "TheBloke/WizardLM-Uncensored-Falcon-7B-GPTQ"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

tokenizer_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/305 [00:00<?, ?B/s]

Make sure that you are connected to a GPU:

In [ ]:
import torch
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU instead.")

Using GPU: NVIDIA L4


In [ ]:
torch_dtype = torch.float16 if device.type == 'cuda' else torch.float32 # change the datatype based on if the gpu type is cuda.

model = AutoGPTQForCausalLM.from_quantized(model_path, device="cuda:0", use_triton=False, torch_dtype = torch_dtype, use_safetensors=True, trust_remote_code=True, disable_exllamav2=True)
model.to(device)

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.
WARNING - can't get model's sequence length from model config, will set to 4096.
WARNING - RWGPTQForCausalLM hasn't fused attention module yet, will skip inject fused attention.
WARNING - RWGPTQForCausalLM hasn't fused mlp module yet, will skip inject fused mlp.


RWGPTQForCausalLM(
  (model): RWForCausalLM(
    (transformer): RWModel(
      (word_embeddings): Embedding(65025, 4544)
      (h): ModuleList(
        (0-31): 32 x DecoderLayer(
          (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
          (self_attention): Attention(
            (maybe_rotary): RotaryEmbedding()
            (query_key_value): QuantLinear()
            (dense): QuantLinear()
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (mlp): MLP(
            (dense_h_to_4h): QuantLinear()
            (act): GELU(approximate='none')
            (dense_4h_to_h): QuantLinear()
          )
        )
      )
      (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=4544, out_features=65025, bias=False)
  )
)

In [ ]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU instead.")

Using GPU: NVIDIA L4


Load your dataframe that consists of toxic examples:

In [ ]:
import pandas as pd

df = pd.read_csv('/content/examples_for_prompt_cleaned.csv') # use your file path.

In [ ]:
# Prompts to try:

text_prompt = (
      "DEFINITION:" f"{DEFINITION}\n\n"
    "Generate 5 offensive social media comments that conforms with the DEFINITION stated above. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH COMMENT INSIDE DOUBLE QUOTES. :Below are a few examples of the kind of offensive comments that you need to generate:\n"
    "example1: \"What a fucking asshole bully. He deserves a long sentence! In reading the article, it's doubtful any friends and family will miss him.\"\n"
      "example2: \"Nsw are such shit they even wrecked a whole other country. Feel so sorry for the nzers\"\n"
      )

text_prompt = (
      "DEFINITION:" f"{DEFINITION}\n\n"
    "Generate 5 offensive social media comments that conforms with the DEFINITION stated above. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH COMMENT INSIDE DOUBLE QUOTES. :Below are a few examples of the kind of offensive comments that you need to generate:\n"
    "example1: \"These scuzzballs will just get on GoFundMe and claim that evil Biden and the Deep State are illegally using the courts to bully them into silence, but they need YOUR HELP to keep exposing DEM CORRUPTION.\"\n"
      "example2: \"As an Arkansan, fuck Asa. What a piece of shit. I've lost two coworkers at the hospital I work at to COVID. Fuck him, and fuck the GOP, and fuck all the ignorant ass hillbilly rejects that perpetuate the misinformation. They're getting people killed.\"\n"
      )

text_prompt = (
      "DEFINITION:" f"{DEFINITION}\n\n"
    "Generate 5 offensive social media comments that conforms with the DEFINITION stated above. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH COMMENT INSIDE DOUBLE QUOTES. :Below are a few examples of the kind of offensive comments that you need to generate:\n"
    "example1: \"I dunno. They do an awful lot of raping for people who don't objectify women\"\n"
      "example2: \"What the hell does the number of patients in hospital have to do with this storm coming? Anything to boost your narrative I guess...shameless media whores you are.\"\n"
  )

text_prompt = (
      "Below are a few examples of the kind of offensive comments that you need to generate:\n"
      "example1: \"What a fucking asshole bully. He deserves a long sentence! In reading the article, it's doubtful any friends and family will miss him.\"\n"
      "example2: \"what do you expect when the rulers of their country are the same rulers who televised beheadings, slaughtered innocent people and were sexist towards women, you make me want to projectile vomit straight in your face 🤢🤮 I feel bad about the fact that you think you’re so clever\"\n"
      "example3: \"Yep, meanwhile Trump is still trying to point out Afghanistan on the map. Illiterate orange clown and his cult. Hilarious you attack biden when bonespurs can't even read at a 3rd grade level.\"\n"
      "DEFINITION:" f"{DEFINITION}\n\n"
      "Generate 5 new offensive social media (Facebook) comments that conforms with the DEFINITION stated above. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH OF THE 5 COMMENTS INSIDE DOUBLE QUOTES AND OF LENGTH IN BETWEEN 20 AND 100 WORDS AND VARY THE LENGTH FOR THE COMMENTS. (NOTE: MAKE SURE TO NOT GENERATE THE SAME COMMENTS AS THE EXAMPLES SHOWN ABOVE)"
  )

text_prompt = (
        "DEFINITION:" f"{DEFINITION}\n\n"
        "Generate 5 offensive social media comments maybe a couple comments on real world politics that conforms with the DEFINITION stated above and include a couple witty ones that would direclty not seem like offensive but is offensive. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE THE COMMENT INSIDE DOUBLE QUOTES"
        )

text_prompt = (
        "DEFINITION:" f"{DEFINITION}\n\n"
        "Generate a social media comment which is both hateful and offensive and that conforms with the DEFINITION stated above. THE GENERATED COMMENT SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE THE COMMENT INSIDE DOUBLE QUOTES"
        )

text_prompt = (
    f"DEFINITION: {DEFINITION}\n\n"
    f"Generate 5 social media comments which are both offensive and that conforms with the DEFINITION stated above. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH OF THE 5 COMMENTS INSIDE DOUBLE QUOTES AND OF LENGTH IN BETWEEN 20 AND 100 WORDS AND VARY THE LENGTH FOR THE COMMENTS"
  )

text_prompt = (
      "DEFINITION:" f"{DEFINITION}\n\n"
    "Generate 5 offensive social media comments that conforms with the DEFINITION stated above. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH COMMENT INSIDE DOUBLE QUOTES. :Below are a few examples of the kind of offensive comments that you need to generate:\n"
    "example1: \"What a fucking asshole bully. He deserves a long sentence! In reading the article, it's doubtful any friends and family will miss him.\"\n"
      "example2: \"Nsw are such shit they even wrecked a whole other country. Feel so sorry for the nzers\"\n"
      )

text_prompt = (
      "DEFINITION:" f"{DEFINITION}\n\n"
    "Generate 5 offensive social media (Facebook) comments that conforms with the DEFINITION stated above. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH COMMENT INSIDE DOUBLE QUOTES GENERATE EACH OF THE 5 COMMENTS INSIDE DOUBLE QUOTES AND OF LENGTH IN BETWEEN 20 AND 100 WORDS AND VARY THE LENGTH FOR THE COMMENT.
      )

# Function to generate the data based on prompting for a given category from the category list

In [ ]:
import json
import requests
import pandas as pd
import time
import re
import torch

def generate_category_data(CATEGORY, model = model):
  model = model.to(device)
  generated_examples = []
  generated_hashtags = []
  start_time = time.time()

  n_examples = 1 # for each of the two examples that you provide in the prompt generate n_examples times using the same prompt.

  DEFINITION = """ OFFENSIVE COMMENTS: contains hurtful, derogatory, or obscene comments """

  # Create the text prompt
  text_prompt = (
      "DEFINITION:" f"{DEFINITION}\n\n"
    "Generate 5 offensive social media comments that conforms with the DEFINITION stated above targeting " f"{CATEGORY}. THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH COMMENT INSIDE DOUBLE QUOTES. :Below are a few examples of the kind of offensive comments that you need to generate:\n"
    "example1: \"What a fucking asshole bully. He deserves a long sentence! In reading the article, it's doubtful any friends and family will miss him.\"\n"
      "example2: \"Nsw are such shit they even wrecked a whole other country. Feel so sorry for the nzers\"\n"
      )

  prompt = text_prompt
  prompt_template = f"### Instruction: {prompt}\n### Response:"
  tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.to(device)

  encoding = tokenizer(prompt_template, return_tensors="pt", padding=True, truncation=True, max_length=512)
  input_ids = encoding.input_ids.to(device)
  attention_mask = encoding.attention_mask.to(device)

  for i in range(n_examples):
    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,  # Pass the attention mask here
        pad_token_id=tokenizer.eos_token_id,  # Set pad_token_id here
        max_new_tokens=256,
        do_sample=True,
        temperature=0.8
    )

    txt=tokenizer.decode(output[0], skip_special_tokens=True)
    #print(txt)

    response_pattern = r"### Response:(.*?)(?=### |$)"

    response_match = re.search(response_pattern, txt, re.DOTALL)
    # print(response_match)

    if response_match:
        response_text = response_match.group(1)
        cleaned_text=response_text.strip() # Use strip() to remove leading/trailing whitespace
            # Remove hashtags and the words attached to them
        cleaned_text = re.sub(r'#\w+', '', cleaned_text)
    else:
        cleaned_text=None

    quoted_text = cleaned_text
    #print("QUOTED TEXT: ", quoted_text)

    if quoted_text == None or quoted_text == "":
      quoted_text = ""

  ####

    hashtags = re.findall(r"#(\w+)", txt)
    hashtags_str = ', #'.join(hashtags)
    generated_examples.extend(re.findall(r'"([^"]*)"', quoted_text))
    generated_hashtags.append(hashtags_str)
    print("Generated output: ", i)

  end_time = time.time()
  print("Finished category: ", CATEGORY)

  print("Total time taken: ", end_time - start_time)

  return generated_examples

# Function to generate data by prompting with examples:

In [ ]:
# the function returns a list of toxic generated examples, (i.e each example is a string)
# df will be the dataframe that you will be passing that consists of toxic examples, the prompt takes in 2 rows each time and prompts
# the model to generate 5 examples.

def generate_data(df, model = model):
    model = model.to(device)
    generated_examples = []
    generated_hashtags = []
    start_time = time.time()

    DEFINITION = """ OFFENSIVE COMMENTS: contains hurtful, derogatory, or obscene comments """

    for i in range(0, len(df), 2):
        example1 = df.iloc[i]['example']
        example2 = df.iloc[i + 1]['example'] if i + 1 < len(df) else ''

        n_examples = 5 # for each of the two examples that you provide in the prompt generate n_examples times using the same prompt.

        # the prompt below asks to generate 5 posts within double quotes, so that we can extract it in a much easier way using regex.
        # you can change your prompts and try different prompt engineering techniques.
        text_prompt = (
            "DEFINITION:" f"{DEFINITION}\n\n"
            "Generate 5 offensive social media comments that conforms with the DEFINITION stated above. "
            "THE GENERATED COMMENTS SHOULD BE WRITTEN IN PLAIN ENGLISH AND GENERATE EACH COMMENT INSIDE DOUBLE QUOTES "
            "GENERATE EACH OF THE 5 COMMENTS INSIDE DOUBLE QUOTES AND OF LENGTH IN BETWEEN 20 AND 100 WORDS AND VARY THE LENGTH FOR THE COMMENT. "
            "Below are a few examples of the kind of offensive comments that you need to generate:\n"
            f"example1: \"{example1}\"\n"
            f"example2: \"{example2}\"\n"
        )

        prompt = text_prompt
        prompt_template = f"### Instruction: {prompt}\n### Response:"
        tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.to(device)

        encoding = tokenizer(prompt_template, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = encoding.input_ids.to(device)
        attention_mask = encoding.attention_mask.to(device)


        for j in range(n_examples):
          output = model.generate(
              input_ids=input_ids,
              attention_mask=attention_mask,
              pad_token_id=tokenizer.eos_token_id,
              max_new_tokens=256,
              do_sample=True,
              temperature=0.8
          )

          txt=tokenizer.decode(output[0], skip_special_tokens=True)
          response_pattern = r"### Response:(.*?)(?=### |$)"
          response_match = re.search(response_pattern, txt, re.DOTALL)

          if response_match:
              response_text = response_match.group(1)
              cleaned_text=response_text.strip() # Use strip() to remove leading/trailing whitespace
                  # Remove hashtags and the words attached to them
              cleaned_text = re.sub(r'#\w+', '', cleaned_text)
          else:
              cleaned_text=None

          quoted_text = cleaned_text
          #print("QUOTED TEXT: ", quoted_text)

          if quoted_text == None or quoted_text == "":
            quoted_text = ""

          hashtags = re.findall(r"#(\w+)", txt)
          hashtags_str = ', #'.join(hashtags)
          generated_examples.extend(re.findall(r'"([^"]*)"', quoted_text)) # extracting text within double quotes
          generated_hashtags.append(hashtags_str)
          #print("Generated output: ", j)

        end_time = time.time()
        print("Finished set: ", i) # printing the number of the set finished, it will all be even numbers since we are using 2 examples at the same time.

        print("Total time taken: ", end_time - start_time) # time taken for inference for each set.

    return generated_examples

In [ ]:
# store the generated data in a list.

data_offensive = generate_data(df)

Finished set:  0
Total time taken:  318.9509971141815
Finished set:  2
Total time taken:  614.3444910049438


# Add the data to a dataframe:

In [ ]:
data_examples_offensive = pd.DataFrame({
    'text': data_offensive,
    'hateful': 0,
    'offensive': 1,
    'toxic': 0,
    'toxicity': 1
}) # change the values to be 0 or 1 based on if you are trying to generate offensive, hateful or toxic text.

# Download the data as a csv file:

In [ ]:
from google.colab import files

data_examples_offensive_4.to_csv('your_filename.csv', index=False)

files.download('your_filename.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Generating data targeting a specific category:

In [ ]:
categories_list = [
    "Muslims",
    "Jews",
    "Sikhs",
    "Buddhists",
    "Transgender individuals",
    "Non-binary individuals",
    "Women",
    "LGBTQ+ individuals",
    "Persons with disabilities",
    "Refugees",
    "Indigenous Peoples",
    "Black/African American",
    "Latinx",
    "Asian Americans/Pacific Islanders"
]

In [ ]:
final_generation = []

for CATEGORY in categories_list:
  final_generation.extend(generate_category_data(CATEGORY))

Generated output:  0
Generated output:  1
Generated output:  2
Generated output:  3
Generated output:  4
Generated output:  5
Generated output:  6
Generated output:  7
Generated output:  8
Generated output:  9
Generated output:  10
Generated output:  11
Generated output:  12
Generated output:  13
Generated output:  14
Finished category:  Muslims
Total time taken:  2400.393069267273
Generated output:  0
Generated output:  1
Generated output:  2
Generated output:  3
Generated output:  4
Generated output:  5
Generated output:  6
Generated output:  7
Generated output:  8
Generated output:  9
Generated output:  10
Generated output:  11
Generated output:  12
Generated output:  13
Generated output:  14
Finished category:  Jews
Total time taken:  2401.563175678253
Generated output:  0
Generated output:  1
Generated output:  2
Generated output:  3
Generated output:  4
Generated output:  5
Generated output:  6
Generated output:  7
Generated output:  8
Generated output:  9
Generated output:  10
G

In [ ]:
data_final_generation = pd.DataFrame({
    'text': final_generation,
    'hateful': 0,
    'offensive': 1,
    'toxic': 0,
    'toxicity': 1
}) # change the values to be 0 or 1 based on if you are trying to generate offensive, hateful or toxic text.

You can further do the same thing as before and download it as a csv file.